In [7]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from enum import Enum
from copy import copy
from scipy.stats import norm
from scipy.cluster import hierarchy as sch
import pandas as pd
import numpy as np

import yfinance as yf

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


from _utils.strategies.trend_simple_filter import (
    apply_buffering_to_position_dict
)
from _utils.strategies.simple_carry import *
from _utils.normalized_price import calculate_normalised_price_dict
from _utils.strategies.trend_asset_classes import (

    calculate_relative_price_dict
    )

from _utils.strategies.breakout import (
    breakout,
    calculate_forecast_for_breakout
)


from _utils.strategies.value import (
    value,
    calculate_forecast_for_value
)

DATA_DIR = "../_databases"

DEFAULT_DATE_FORMAT = "%Y-%m-%d"


In [8]:
#%% GET DATA
# # =================

def get_data_dict(instrument_list: list):

    all_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]
    )

    adjusted_prices = dict(
        [
            (instrument_code, data_for_instrument.adjusted)
            for instrument_code, data_for_instrument in all_data.items()
        ]   
    )

    current_prices = dict(
        [
            (instrument_code, data_for_instrument.underlying)
            for instrument_code, data_for_instrument in all_data.items()
        ]
    )
    
    carry_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '_carry.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]   
    )

    return adjusted_prices, current_prices, carry_data


def get_fx_data_dict(instrument_list: list):

    all_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]
    )

    price = dict(
        [
            (instrument_code, data_for_instrument.price)
            for instrument_code, data_for_instrument in all_data.items()
        ]   
    )


    return price

In [9]:
# função auxiliar
def align_dict_to_index(data_dict, target_index):
    for inst in data_dict.keys():
        df = data_dict[inst]

        data_dict[inst] = df.reindex(target_index).ffill().bfill()

    return data_dict




In [10]:
adjusted_prices_dict, current_prices_dict, carry_prices_dict = get_data_dict(["sp500", "eurostx", "us10", "us2"])

ref = "eurostx"

target_index = adjusted_prices_dict[ref].index
adjusted_prices_dict  = align_dict_to_index(adjusted_prices_dict,  target_index)
current_prices_dict   = align_dict_to_index(current_prices_dict,   target_index)
carry_prices_dict     = align_dict_to_index(carry_prices_dict,     target_index)
fx_data_dict = get_fx_data_dict(['eur_fx'])
multipliers = dict(sp500=5, eurostx=10, us10=1000, us2=2000)
risk_target_tau = 0.2
asset_class_groupings = dict(bonds=["us2", "us10"], stocks=["sp500", "eurostx"])

fx_series_dict = create_fx_series_given_adjusted_prices_dict(adjusted_prices_dict,dict(sp500=1, eurostx=fx_data_dict['eur_fx'], us10=1, us2=1))

capital = 2000000
idm = 2
instrument_weights = dict(sp500=0.25, eurostx=0.25, us10=0.25, us2=0.25)
cost_per_contract_dict = dict(sp500=0.875, eurostx=6.8, us10=9.5, us2=5.5)


std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
adjusted_prices=adjusted_prices_dict,
current_prices=current_prices_dict,
annualise_stdev=True,  ## can also be False if want to use daily price diff
use_perc_returns=True,  ## can also be False if want to use daily price diff
)

In [11]:
average_position_contracts_dict = (
        calculate_position_series_given_variable_risk_for_dict(
            capital=capital,
            risk_target_tau=risk_target_tau,
            idm=idm,
            weights=instrument_weights,
            std_dev_dict=std_dev_dict,
            fx_series_dict=fx_series_dict,
            multipliers=multipliers,
        )
    )

normalised_price_dict = calculate_normalised_price_dict(
       adjusted_prices_dict=adjusted_prices_dict,
       std_dev_dict=std_dev_dict,
   )


rules_spec_ewmac = [
        dict(function="ewmac", fast_span=16),
        dict(function="ewmac", fast_span=32),
        dict(function="ewmac", fast_span=64),
    ]

list_of_rules_breakout = [
       dict(function=breakout, scalar=0.6, horizon=10),
       dict(function=breakout, scalar=0.67, horizon=20),
       dict(function=breakout, scalar=0.7, horizon=40),
       dict(function=breakout, scalar=0.73, horizon=80),
       dict(function=breakout, scalar=0.74, horizon=160),
       dict(function=breakout, scalar=0.74, horizon=320),
   ]

list_of_rules_value = [dict(function=value, horizon_years=5, scalar=7.27, asset_class_groupings=asset_class_groupings)]



position_contracts_value_dict = calculate_position_dict_with_forecast_from_function_applied(
        adjusted_prices_dict=adjusted_prices_dict,
        carry_prices_dict=carry_prices_dict,
        std_dev_dict=std_dev_dict,
        average_position_contracts_dict=average_position_contracts_dict,
        list_of_rules=list_of_rules_value,
    )


buffered_position_value_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_value_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_value_dict = calculate_perc_returns_for_dict(
    position_contracts_dict=buffered_position_value_dict,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
)


port_value_return = aggregate_returns(
    perc_return_value_dict)

port_value_return.sort_index(inplace=True)

#=====================================

position_contracts_breakout_dict = calculate_position_dict_with_forecast_from_function_applied(
        adjusted_prices_dict=adjusted_prices_dict,
        carry_prices_dict=carry_prices_dict,
        std_dev_dict=std_dev_dict,
        average_position_contracts_dict=average_position_contracts_dict,
        list_of_rules=list_of_rules_breakout,
    )


buffered_position_breakout_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_breakout_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_breakout_dict = calculate_perc_returns_for_dict(
    position_contracts_dict=buffered_position_breakout_dict,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
)


port_breakout_return = aggregate_returns(
    perc_return_breakout_dict)

port_breakout_return.sort_index(inplace=True)

In [12]:
# --- Plot retorno acumulado
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=port_value_return.index,
        y=port_value_return.cumsum()*100,
        name="Value",
        line=dict(color="blue"),
    )
)


fig.add_trace(
    go.Scatter(
        x=port_breakout_return.index,
        y=port_breakout_return.cumsum()*100,
        name="Breakout",
        line=dict(color="Red"),
    )
)




fig.update_layout(
    title="Strategies for Future Contracts Portfolio",
    xaxis_title="Data",
    yaxis_title="Acumulated return (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=0.8,
        xanchor="left",
        x=0.0),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho")
print("Estatísticas Portfolio Value Frequência Anual")
print(calculate_stats(port_value_return, freq=YEAR))
print("================================")
print("Estatísticas Portfolio Breakout trend Frequência Anual")
print(calculate_stats(port_breakout_return, freq=YEAR))
print("================================")


Estatísticas de Desempenho
Estatísticas Portfolio Value Frequência Anual
{'ann_mean': -9.000318167087649, 'ann_std': 2.271688383755971, 'sharpe': -3.9619510455068117, 'skew': -0.8193891170787287, 'avg_drawdown': 0.058717830247371726, 'max_drawdown': 0.3392985076904297, 'quant_lower': 0.49113422249630345, 'quant_upper': 0.6864848967591185}
Estatísticas Portfolio Breakout trend Frequência Anual
{'ann_mean': 24.218487220042448, 'ann_std': 3.208862549854376, 'sharpe': 7.547374449286251, 'skew': 1.7140819409998225, 'avg_drawdown': 0.02626298801131847, 'max_drawdown': 0.09013237308906251, 'quant_lower': 0.2386335196889049, 'quant_upper': 2.757692940507002}
